# 모델은 왜 그렇게 예측하였을까? SHAP

## 1.환경준비

### 1) 라이브러리 설치 및 로딩

In [ ]:
!pip install shap

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from xgboost import XGBRegressor, plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR

import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
tf.compat.v1.disable_v2_behavior() # shap 그래프가 tf1 버전을 지원합니다. tf2는 비활성화.

import shap

### 2) 데이터 로딩

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/boston.csv')
data.head()

* 변수 설명
    * crim : 범죄율
    * zn : 대저택 비율
    * indus : 상업지역 비율
    * chas : 찰스 강변 여부
    * nox : 일산화질소 농도(공기오염도)
    * rm : 평균 주택당 방 수
    * age : 30년 이상된 주택 비율
    * dis : 주요 업무 지역 접근성 지수
    * rad : 고속도로 접근성 지수
    * tax  1만 달러당 재산세
    * ptratio : 교사1명당 학생수
    * lstat : 하위계층 비율
    * black : 흑인비율(원래 숫자를 변형한 것임)
    * mdev : 타운별 집값 중위수(단위 : 1000달러)

### 3) 필요한 함수 

In [ ]:
def plot_feature_importance(importance, names, topn = 'all'):
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    plt.figure(figsize=(10,8))
    sns.barplot(x='feature_importance', y='feature_names', data = fi_df)

    plt.xlabel('importance')
    plt.ylabel('feature names')
    plt.grid()

    return fi_df

## 2.데이터 준비

### 1)데이터분할

In [ ]:
target = 'medv'
x = data.drop(target, axis=1)
y = data[target]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .2, random_state = 10)

### 2)스케일링
* DL 모델과 SVM 모델에 사용

In [ ]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

# x_train_s = pd.DataFrame(x_train_s, columns = list(x_train))
# x_val_s = pd.DataFrame(x_val_s, columns = list(x_val))

## 3.모델링 및 해석

* RandomForest


### 1) 학습

In [ ]:
model1 = RandomForestRegressor()
model1.fit(x_train, y_train)

### 2) 해석

#### ① Shapley Value 만들기

In [ ]:
# SHAP 값으로 모델의 예측 설명하기
explainer1 = shap.TreeExplainer(model1)
shap_values1 = explainer1.shap_values(x_train)

In [ ]:
x_train.shape, shap_values1.shape

In [ ]:
x_train[:2]

In [ ]:
pd.DataFrame(shap_values1, columns = list(x))[:2]

#### ② 특정 데이터에 대한 설명

In [ ]:
x_train.iloc[0:1,:]

* 예측값

In [ ]:
# 예측하기 위해서는 입력데이터(x)가 2차원이어야 합니다.
pred = model1.predict(x_train.iloc[0:1,:])
pred, pred[0]

* **train의 예측값 전체 평균**
    * 보통, 실제값의 평균(y_train.mean())과 예측값의 평균(pred.mean())은 거의 같습니다.

In [ ]:
model1.predict(x_train).mean()

* 예측값과 y평균간 차

In [ ]:
pred[0] - model1.predict(x_train).mean()

* Shapley Value

In [ ]:
pd.DataFrame(shap_values1[0:1, :], columns = list(x_train))

* Shapley Value 합 : 
    * 소수점 반올림된 값 때문에 약간 차이가 있음(넘어갑시다.^^)
    * 또한 랜덤포레스트 모델일 경우, 알고리즘의 특성상 약간의 차이가 발생될 수 있답니다.  
        * https://github.com/slundberg/shap/issues/318#issuecomment-437429986 

In [ ]:
shap_values1[0:1, :].sum()

* 그래프로 그려 봅시다.

In [ ]:
# y_train의 평균
explainer1.expected_value

In [ ]:
shap.initjs() # javascript 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.

# force_plot(전체평균, shapley_values, input)
shap.force_plot(explainer1.expected_value, shap_values1[0, :], x_train.iloc[0,:])

* 집값 형성에 대해서, 예측결과가 결정 되는데, 
    * 상승 요인 : crim, ptratio, tax, nox, age
    * 하락 요인 : rm, dis, lstat, indus

## 4.실습

* 모델의 전체 중요도로 볼 때, lstat가 집값에 가장 큰 영향을 줍니다.
* lstat와 집값(medv)의 관계를 살펴보면, 강한 음의 상관관계가 있음을 알수 있습니다. 
* 그러나, 이 관계에서 벗어난 데이터들이 존재합니다. 이들에 대해서 집값을 예측하는데 영향을 주는 요인을 각각 찾아 설명해 봅시다.

### 1) 모델의 변수 중요도 

### 2) 전체 흐름에서 벗어나 보이는 값들 분석하기

* 아래 그래프를 보면 index 371, 266, 505, 214 데이터가 전체 흐름과 다른 집값을 나태내고 있습니다. 
* 이 값들에 대해서 shapley value를 확인하고 그래프를 통해 이유를 설명해 봅시다.

In [ ]:
plt.figure(figsize = (10,6))
sns.scatterplot(x='lstat', y = 'medv', data = data)

sns.scatterplot(x='lstat', y = 'medv', data = data.iloc[[371]], color = 'r')
plt.text(10.5, 49.6, '371(9.53, 50)', color = 'r')
sns.scatterplot(x='lstat', y = 'medv', data = data.iloc[[266]], color = 'r')
plt.text(15.5, 30.2, '266(14.79,  30.7)', color = 'r')
sns.scatterplot(x='lstat', y = 'medv', data = data.iloc[[505]], color = 'r')
plt.text(8.5, 11.5, '505(7.88,  11.9)', color = 'r')
sns.scatterplot(x='lstat', y = 'medv', data = data.iloc[[214]], color = 'r')
plt.text(30.2, 23.3, '214(29.55,  23.7)', color = 'r')

plt.grid()
plt.show()

In [ ]:
# SHAP 값으로 모델의 예측 설명하기
shap_values1 = explainer1.shap_values(x)

* index = 371

In [ ]:
data.iloc[[371]]

In [ ]:
shap_values1[371:372, :]

In [ ]:
shap.initjs() # JS 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.

# force_plot(전체평균, shapley_values, input)



> * lstat = 9.53 ==> 실제 값
* 화살표의 길이 ==> shapley value, 기여도


* index = 266

In [ ]:
shap.initjs() # JS 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.

# force_plot(전체평균, shapley_values, input)



* index = 505

In [ ]:
shap.initjs() # JS 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.

# force_plot(전체평균, shapley_values, input)



* index = 214

In [ ]:
shap.initjs() # JS 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.

# force_plot(전체평균, shapley_values, input)



## 5.추가 분석

### 1) 데이터 전체를 한눈에

#### ① 전체 변수에 대한 shap value 시각화
* 각 변수별 shapley value 분포를 한눈에 보여줍니다.
* 색깔은 해당 변수의 값의 크기를 나타냅니다.

In [ ]:
shap_values1 = explainer1.shap_values(x_train)

In [ ]:
shap.summary_plot(shap_values1, x_train)

#### ② 특정 관점으로 Data point이 Shap Value 정렬

In [ ]:
shap.initjs()
shap.force_plot(explainer1.expected_value, shap_values1, x_train)

#### ③ 특정 변수 값과 변수의 shap value 간의 관계

In [ ]:
shap.dependence_plot('lstat', shap_values1, x_train)

In [ ]:
shap.dependence_plot('lstat', shap_values1, x_train, interaction_index = 'nox')

-----------
아래 내용은 시간이 많이 걸리기에 **참조 자료** 로 보시면 좋겠습니다.^^

### 2) KernelExplainer

* SVM에 대한 shapley value 계산은 엄청 오래 걸립니다.
* 그래서 데이터를 줄여서 수행합니다.

#### ① SVM 모델링

In [ ]:
model2 = SVR()
model2.fit(x_train_s, y_train)

#### ② Shap Value 만들기
* shap.KernelExplainer(모델.**predict**, x)
    * TreeExplainer는 그냥 모델을 넣었지만, KernelExpaliner는 모델.**predict**로 넣어야 합니다.
* 데이터를 줄여서 10건만 수행합니다.

In [ ]:
explainer2 = shap.KernelExplainer(model2.predict, x_train_s)

# 10건만 줄여서 수행합니다. 
shap_values2 = explainer2.shap_values(x_train_s[:10])

In [ ]:
shap_values2

In [ ]:
shap.initjs()
shap.force_plot(explainer2.expected_value, shap_values2, x_train_s[:10])

In [ ]:
shap.initjs()
shap.force_plot(explainer2.expected_value, shap_values2[0], x_train.iloc[0])

### 3) DeepExplainer

#### ① DL 모델링

In [ ]:
# 간단한 DL 모델을 생성합니다.
nfeatures = x_train.shape[1]

clear_session()

model3 = Sequential([Dense(4, input_shape = (nfeatures,), activation = 'relu'),
                     Dense(1)])

model3.compile(optimizer='adam', loss='mse')

history = model3.fit(x_train_s, y_train, 
                     epochs = 100, batch_size = 64,
                    validation_split=0.2).history

#### ② Shap Value 만들기

* DeepExplainer로 부터 shap_values를 추출하면
    * **리스트 안에 np.array로 값이 저장**됩니다.(아니..왜...)
    * 그래서 이를 사용하려면 값을 뽑아내서(리스트[0])로 사용해야 합니다.

In [ ]:
explainer = shap.DeepExplainer(model3, x_train_s)
shap_values = explainer.shap_values(x_train_s)

In [ ]:
shap_values

In [ ]:
# 리스트에서 값을 빼내기
shap_values = shap_values[0]

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, x_train)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], x_train.iloc[0])

In [ ]:
shap.summary_plot(shap_values, x_train)

## 6.정리(아직 일관성 떨어지는 SHAP 사용하기 위해)

* shap.TreeExplainer(model1)
* shap.KernelExplainer(모델.predict, x)
* shap.DeepExplainer(model3, x_train_s)
* knn : shap.KernelExplainer(모델.predict_proba, x)